In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
image_size = [224,224]
#data_path = 'Data/Training'

In [3]:
vgg = VGG16(input_shape= image_size+[3],weights='imagenet',include_top=False)

In [4]:

vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
vgg.output

<tf.Tensor 'block5_pool/Identity:0' shape=(None, 7, 7, 512) dtype=float32>

In [6]:
x = vgg.output
x = GlobalAveragePooling2D()(x)

In [7]:
x = Dense(1024,activation='relu')(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512, activation='relu')(x)
x= Dense(256,activation='relu')(x)

In [8]:
preds = Dense(2,activation='softmax')(x)

In [9]:
model = Model(inputs = vgg.input,outputs=preds)

In [11]:
#\model.summary()
#from keras.utils import plot_model
#plot_model(model, to_file='model_new.png')

In [ ]:
# for layer in vgg.layers[:20]:
#     layer.trainable=False
# for layer in vgg.layers[20:]:
#     layer.trainable = True

In [12]:
for layer in vgg.layers:
    layer.trainable = False

In [15]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('C:\\Users\\Soumyabroto Banerjee\\Desktop\\Data\\Training\\', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=35,
                                                 class_mode='categorical',
                                                 shuffle=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
        'C:\\Users\\Soumyabroto Banerjee\\Desktop\\Brain_Tumor_Classification\\Data\\Test',
        target_size=(224, 224),
        color_mode='rgb',
        batch_size=18,
        class_mode='binary',shuffle=False)

Found 280 images belonging to 2 classes.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Soumyabroto Banerjee\\Desktop\\Brain_Tumor_Classification\\Data\\Test'

In [ ]:
#x,y = test_generator.next()
#for i in range(0,1):
#    imae = x[i]
#    plt.imshow(imae.transpose(2,1,0))
#    plt.show()

In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(train_generator.class_indices,np.unique(train_generator.labels,return_counts=True))
print(test_generator.class_indices,np.unique(test_generator.labels,return_counts=True))

In [ ]:
print(train_generator.n)
print(train_generator.batch_size)
print(train_generator.n//train_generator.batch_size)

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size
r = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

In [ ]:
step_size_valid=test_generator.n//test_generator.batch_size
scores=model.evaluate_generator(generator=test_generator,
steps=step_size_valid)
print('Loss =, Accuracy = ',scores[0],scores[1])

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')


In [ ]:
from tensorflow.keras.models import load_model
model.save('brain_tumor2.h5')

In [ ]:
######################## PREDICTION #########################

In [ ]:
step_size_test=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=step_size_test,
verbose=1)

In [ ]:
predicted=np.argmax(pred,axis=1)
actual=test_generator.classes

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(predicted,actual))

In [ ]:
######################## PREDICTION of 1 individual image #########################

In [ ]:
img_path = 'Data/Training/no/15 no.jpg'
img = image.load_img(img_path,target_size=(224,224))
x1= image.img_to_array(img)
x = np.expand_dims(x1,axis=0)
img_data = preprocess_input(x)

In [ ]:
rslt = model.predict(img_data)
print(rslt)

In [ ]:
if rslt[0][0] == 1:
    prediction = 'Not A Brain Tumor'
else:
    prediction = 'Brain Tumor'
    
print(prediction)